In [ ]:
%pprint

In [ ]:
def tokenize_words(words, encode_dict): # words: ["ahoi", "matey", ..."]
    sorted_tokens = sorted(encode_dict.items(), key=lambda item: len(item[0]), reverse=True)
    token_ids_array = []
    for w in words:
        token_ids = []
        ptr = 0
        while ptr < len(w):
            for token, idx in sorted_tokens:
                if w[ptr:].startswith(token):
                    token_ids.append(idx)
                    ptr += len(token)
                    break

        token_ids_array.append(token_ids)
    return token_ids_array

def bigram_counts(word_token_ids): # elements: [[1, 4, 22], [2, 44, 0], ...]
    bigram_counts = {}

    for tokens in word_token_ids:
        for a, b in zip(tokens, tokens[1:]):
            bigram_counts[(a, b)] = bigram_counts.get((a, b), 0) + 1

    return bigram_counts

# txt="""
# When Aunt Em came there to live she was a young, pretty wife. The sun
# and wind had changed her, too. They had taken the sparkle from her eyes
# and left them a sober gray; they had taken the red from her cheeks and
# lips, and they were gray also. She was thin and gaunt, and never smiled
# now. When Dorothy, who was an orphan, first came to her, Aunt Em had
# """
with open("./wizard-of-oz.txt", encoding="utf-8-sig") as f:
    txt = f.read()
chars = sorted(list(set(txt)))
encode_dict = {c:i for i,c in enumerate(chars)}
decode_dict = {i:c for c,i in encode_dict.items()}

words = sorted(list(set(txt.split(" "))))
if "" in words:
    words.remove("")

for _ in range(1000):
    word_token_ids = tokenize_words(words, encode_dict)
    if all([len(ids) == 1 for ids in word_token_ids]):
        print("\n\nexhausted all words")
        break

    counts = bigram_counts(word_token_ids)
    most_common_bigram_idx = sorted(counts.items(), key=lambda item: item[1], reverse=True)[0][0]
    most_common_bigram = "".join([decode_dict[idx] for idx in most_common_bigram_idx])

    encode_dict[most_common_bigram] = len(encode_dict)
    decode_dict = {i:c for c,i in encode_dict.items()}

    print(f"{most_common_bigram!r}, ", end="", flush=True)


In [ ]:
# txt="""
# When Aunt Em came there to live she was a young, pretty wife. The sun
# and wind had changed her, too. They had taken the sparkle from her eyes
# and left them a sober gray; they had taken the red from her cheeks and
# lips, and they were gray also. She was thin and gaunt, and never smiled
# now. When Dorothy, who was an orphan, first came to her, Aunt Em had
# """
with open("./wizard-of-oz.txt", encoding="utf-8-sig") as f:
    txt = f.read()
chars = sorted(list(set(txt)))
encode_dict = {c:i for i,c in enumerate(chars)}
decode_dict = {i:c for c,i in encode_dict.items()}

words = sorted(list(set(txt.split(" "))))
if "" in words:
    words.remove("")

word_token_ids = tokenize_words(words, encode_dict)
for _ in range(10000):
    counts = bigram_counts(word_token_ids)
    new_bigram_tuple = sorted(counts.items(), key=lambda item: item[1], reverse=True)[0][0] # (1, 4)
    new_token_idx = len(encode_dict)

    for word_ in word_token_ids:
        ii = 0
        while ii < len(word_) - 1:
            if word_[ii] == new_bigram_tuple[0] and word_[ii+1] == new_bigram_tuple[1]:
                word_[ii:ii+2] = [new_token_idx]
                ii += 1
            ii += 1

    # print(new_bigram_tuple)
    most_common_bigram = "".join([decode_dict[idx] for idx in new_bigram_tuple])
    if encode_dict.get(most_common_bigram, None) is not None:
        print(f"{most_common_bigram=} existiert schon ({new_bigram_tuple=})")
    encode_dict[most_common_bigram] = new_token_idx
    decode_dict = {i:c for c,i in encode_dict.items()}

    if ii % 100 == 0:
        print("\r"*100 + f"{ii:}, {most_common_bigram!r}, ", end="", flush=True)


In [ ]:
decode_dict[43]

In [ ]:
list(decode_dict.keys())

In [ ]:
if encode_dict.get(" ", None) is None:
    encode_dict[" "] = len(encode_dict)
    decode_dict[encode_dict[" "]] = " "

In [ ]:

def tok_encode(txt):
    sorted_tokens = sorted(encode_dict.items(), key=lambda item: len(item[0]), reverse=True)
    token_ids = []
    ptr = 0
    while ptr < len(txt):
        for token, idx in sorted_tokens:
            if txt[ptr:].startswith(token):
                token_ids.append(idx)
                ptr += len(token)
                break

    return token_ids

def tok_decode(tok_ids):
    return "".join([decode_dict[i] for i in tok_ids])

tok_decode(tok_encode("Is this the real life?"))